In [17]:
!sudo -H pip3 install lxml stemming

    100% |################################| 5.8MB 5.7MB/s eta 0:00:01
    100% |################################| 1.4MB 8.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for stemming ... done
  Stored in directory: /root/.cache/pip/wheels/e8/05/2e/2ddeb64d4464b854b48323f9676528c17560da7d153db7b0e2
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built stemming nltk


In [26]:
import re
import lxml.html
from stemming.porter2 import stem

In [27]:
desc = 'Overview</h2><br/><br/>Seeking a Staff Experimentation Data Scientist to join Intuits Experimentation Center of Excellence (COE) for Small Business Self-Employed Group (SBSEG). We are looking for a creative problem solver with a passion for delivering data-driven insights, a deep knowledge of Testing and Experimentation to optimize SBSEGs digital experiences.<br/><br/>Responsibilities</h2><br/><br/>In this role you will drive experimentation process improvement and automation to dramatically transform the way in which we allocate customers to experiments and manage delivery of experiments that drive decision making.\n<br/><br/>\nResponsibilities:\n\n<ul>\n<li>Work closely with the Embedded Data Scientists/Analysts to set up, maintain and optimize performance analysis of our various in-market experiments</li>\n<li>Provide test requesters with testing methodology and frameworks to create and monitor tests over time</li>\n<li>Work with experimentation tools team to improve & enhance tool capabilities</li>\n<li>Develop traffic forecasts and size opportunities based on constraints and interpretation of data findings</li>\n<li>Define, implement and standardize metrics, reports and dashboards leveraging Tableau or other data visualization tools</li>\n<li>Deliver key metrics, reports, dashboards and ad-hoc analyses with interpretation, contributing to the development of hypotheses and actions</li>\n<li>Work with Marketing and Product Managers to develop learning plans with recommendations of analytics approaches to address questions or validate hypotheses</li>\n<li>Generate follow-up questions with stakeholders, refine data findings and interpret results to drive data-based insights</li>\n<li>Collaborate with internal and external partners to assist with data collection and reporting</li>\n</ul>\n\nQualifications</h2>\n\n\n<ul>\n<li>7-10 years of experience in decision support and site optimization for an online/e-commerce business</li>\n<li>Deep knowledge of testing and web analytics</li>\n<li>Confident applying appropriate analysis methods of causal inference in both experimental and non-experimental situations</li>\n<li>Subject matter expertise with clickstream data, SQL & Hive is a must</li>\n<li>Understanding of complex web ecosystems, best practices and ability to put this knowledge into action</li>\n<li>Ability to tell stories with data, educate effectively, and instill confidence, motivating stakeholders to act on recommendations</li>\n<li>Excellent problem solving skills and end to end quantitative thinking</li>\n<li>Outstanding communications skills with both technical and non-technical colleagues.</li>\n<li>Experience with Tableau or another reporting solution a plus</li>\n<li>Masters or PhD in a quantitative field such as statistics, mathematics, or computer science</li>\n</ul>'

In [38]:
def parse_html(html_string):
    elem = lxml.html.document_fromstring(html_string)
    for text in (x.strip() for x in elem.xpath('//text()') if x.strip()):
        yield text
        
def tokenizer(text, pattern=r'\W+', toLowercase=True):
    pattern = re.compile(pattern)
    if toLowercase:
        text = text.lower()
    for word in pattern.split(text):
        if word:
            yield word
        
def preproc(html_string):
    for text in parse_html(html_string=desc):
        for word in tokenizer(text):
            yield stem(word)

In [39]:
for text in preproc(html_string=desc):
    print(text)

overview
seek
a
staff
experiment
data
scientist
to
join
intuit
experiment
center
of
excel
coe
for
small
busi
self
employ
group
sbseg
we
are
look
for
a
creativ
problem
solver
with
a
passion
for
deliv
data
driven
insight
a
deep
knowledg
of
test
and
experiment
to
optim
sbseg
digit
experi
respons
in
this
role
you
will
drive
experiment
process
improv
and
autom
to
dramat
transform
the
way
in
which
we
alloc
custom
to
experi
and
manag
deliveri
of
experi
that
drive
decis
make
respons
work
close
with
the
embed
data
scientist
analyst
to
set
up
maintain
and
optim
perform
analysi
of
our
various
in
market
experi
provid
test
request
with
test
methodolog
and
framework
to
creat
and
monitor
test
over
time
work
with
experiment
tool
team
to
improv
enhanc
tool
capabl
develop
traffic
forecast
and
size
opportun
base
on
constraint
and
interpret
of
data
find
defin
implement
and
standard
metric
report
and
dashboard
leverag
tableau
or
other
data
visual
tool
deliv
key
metric
report
dashboard
and
ad
hoc
analys
wit

In [22]:
(x.strip() for x in text if x.strip())

<generator object <genexpr> at 0x7f83c40964c0>

In [16]:
etree.XPath("//text()")(lxml.html.document_fromstring(desc))

['Overview',
 'Seeking a Staff Experimentation Data Scientist to join Intuits Experimentation Center of Excellence (COE) for Small Business Self-Employed Group (SBSEG). We are looking for a creative problem solver with a passion for delivering data-driven insights, a deep knowledge of Testing and Experimentation to optimize SBSEGs digital experiences.',
 'Responsibilities',
 'In this role you will drive experimentation process improvement and automation to dramatically transform the way in which we allocate customers to experiments and manage delivery of experiments that drive decision making.\n',
 '\nResponsibilities:\n\n',
 '\n',
 'Work closely with the Embedded Data Scientists/Analysts to set up, maintain and optimize performance analysis of our various in-market experiments',
 '\n',
 'Provide test requesters with testing methodology and frameworks to create and monitor tests over time',
 '\n',
 'Work with experimentation tools team to improve & enhance tool capabilities',
 '\n',
 '

In [65]:
%%file mr-test.py

import json

from mrjob.job import MRJob
from mrjob.protocol import JSONValueProtocol

class MRWordFrequencyCount(MRJob):
    
    OUTPUT_PROTOCOL = StandardJSONProtocol
    
    def mapper(self, _, value):
        item = json.loads(value)
        yield item.get('jobId'), item.get('description', '')

        
if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting mr-test.py


In [66]:
!python3 mr-test.py ../data/job-data-2018-09-08-00-00-37-0b05248a-ef10-4cb6-83cf-c3a3b35d5cdf.txt --output-dir out

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/mr-test.hadoop.20180912.165716.370666
job output is in out
Removing temp directory /tmp/mr-test.hadoop.20180912.165716.370666...


In [36]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [37]:
ps = PorterStemmer()
 
sentence = "gaming, the gamers play games"
words = word_tokenize(sentence)
 
for word in words:
    print(word + ":" + ps.stem(word))

gaming:game
,:,
the:the
gamers:gamer
play:play
games:game


In [40]:
from stemming.porter2 import stem

In [70]:
for word in word_tokenize('Seeking a Staff Experimentation Data Scientist to join Intuits Experimentation Center of Excellence (COE) for Small Business Self-Employed Group (SBSEG). We are looking for a creative problem solver with a passion for delivering data-driven insights, a deep knowledge of Testing and Experimentation to optimize SBSEGs digital experiences.'.lower()):
    print(word + ":" + stem(word))

seeking:seek
a:a
staff:staff
experimentation:experiment
data:data
scientist:scientist
to:to
join:join
intuits:intuit
experimentation:experiment
center:center
of:of
excellence:excel
(:(
coe:coe
):)
for:for
small:small
business:busi
self-employed:self-employ
group:group
(:(
sbseg:sbseg
):)
.:.
we:we
are:are
looking:look
for:for
a:a
creative:creativ
problem:problem
solver:solver
with:with
a:a
passion:passion
for:for
delivering:deliv
data-driven:data-driven
insights:insight
,:,
a:a
deep:deep
knowledge:knowledg
of:of
testing:test
and:and
experimentation:experiment
to:to
optimize:optim
sbsegs:sbseg
digital:digit
experiences:experi
.:.


In [69]:
word_tokenize('Seeking a Staff Experimentation Data Scientist to join Intuits Experimentation Center of Excellence (COE) for Small Business Self-Employed Group (SBSEG). We are looking for a creative problem solver with a passion for delivering data-driven insights, a deep knowledge of Testing and Experimentation to optimize SBSEGs digital experiences.'.lower())

['seeking',
 'a',
 'staff',
 'experimentation',
 'data',
 'scientist',
 'to',
 'join',
 'intuits',
 'experimentation',
 'center',
 'of',
 'excellence',
 '(',
 'coe',
 ')',
 'for',
 'small',
 'business',
 'self-employed',
 'group',
 '(',
 'sbseg',
 ')',
 '.',
 'we',
 'are',
 'looking',
 'for',
 'a',
 'creative',
 'problem',
 'solver',
 'with',
 'a',
 'passion',
 'for',
 'delivering',
 'data-driven',
 'insights',
 ',',
 'a',
 'deep',
 'knowledge',
 'of',
 'testing',
 'and',
 'experimentation',
 'to',
 'optimize',
 'sbsegs',
 'digital',
 'experiences',
 '.']